In [1]:
import pandas as pd
import numpy as np
import cupy as cp
import chainer
from chainer.backends import cuda
from chainer import Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
from chainer import initializers
import codecs

import MeCab
from gensim.models import KeyedVectors
from tqdm import tqdm
import pickle
from sklearn.model_selection import train_test_split
import pandas as pd
import re
import sys
from sklearn import metrics

In [72]:
text_label = pd.read_csv('./text_label_30k.csv')
text_label.columns = ["text","label"]
tagger = MeCab.Tagger('-Owakati -d /home/machi52/.local/lib/mecab/dic/mecab-ipadic-neologd')
tagger.parse("")

'\n'

In [44]:
term_vec    = pickle.loads(open('../leam/paper_default/vectors/glove/glove_vectors.pkl', 'rb').read())

In [8]:
text_label = pd.read_csv('./text_label_600k_nolabel.csv')
text_label.columns = ["text","label"]
tagger = MeCab.Tagger('-Owakati -d /home/machi52/.local/lib/mecab/dic/mecab-ipadic-neologd')
tagger.parse("")
text_label = text_label.drop(122986)
text_label = text_label.drop(257158)
text_label = text_label.drop(268543)
text_label = text_label.drop(275125)
text_label = text_label.drop(325942)
text_label = text_label.drop(333330)
text_label = text_label.drop(341130)
text_label = text_label.drop(374990)
text_label = text_label.drop(375870)
text_label = text_label.drop(421640)
text_label = text_label.drop(462511)
text_label = text_label.drop(481845)
text_label = text_label.drop(500022)
text_label = text_label.drop(547719)
text_label = text_label.drop(591953)

##
# term_vec    = pickle.loads(open('../all_pkl/term_vec_4500k.pkl', 'rb').read())
term_vec    = pickle.loads(open('../leam/paper_default/vectors/glove/glove_vectors.pkl', 'rb').read())

##char
# term_vec    = pickle.loads(open('../all_pkl/char_vec_4500k.pkl', 'rb').read()) 
# term_vec = pickle.loads(open("../all_pkl/word_and_char_vec_4500k.pkl","rb").read())

In [73]:
texts = text_label.text.apply(lambda x: tagger.parse(str(x)).split()).values
word2id = {}
for text in texts:
    for word in text:
        if word in word2id:
            continue
        word2id[word] = len(word2id)
id2word = {}
for key, value in word2id.items():
    id2word[value] = key
label2id = {}
for i, label in enumerate(text_label.label.value_counts().keys()):
    label2id[label] = i
labels = text_label.label.apply(lambda x: x)    

In [74]:
embedding_vectors = np.random.uniform(-0.01, 0.01, (len(word2id), 300))
embedding_vectors = np.load("./paper_default/data/text_emb_30k.npz")["arr_0"]

In [75]:
class_name = ["😊","😵","😡","😭","😱","😩"]
# ["😵","😊",""😭","😱","😩","😡"]
symbol_list = ['（','）','、']
for label_words in class_name:
    for symbol in symbol_list:
        if symbol in label_words:
            label_words.remove(symbol)
    
value_list = [ [ term_vec[i] for i in l] for l in class_name]
value_mean_emo = np.array([ np.mean(l, axis=0)  for l in value_list])

In [76]:
id_texts = [[word2id[word] for word in text]for text in texts]
f = lambda x: np.pad(x, pad_width=(0, 305-len(x)), mode='constant', constant_values=-1)
id_texts = np.array(list(map(f, id_texts)), np.int32)

In [77]:
labels_multi = [np.array(re.sub("\[|\]","",label).split(","),dtype="float64") for label in labels]
labels_multi = np.array(labels_multi,dtype='int32')
train_x, test_x, train_y, test_y = train_test_split(id_texts, labels_multi, random_state=0)

In [90]:
##default
class LEAM(chainer.Chain):
    
    def __init__(self, vocab_size, n_class=6,filter_size=55):
        super(LEAM, self).__init__()
        self.n_class = n_class
        with self.init_scope():
            self.embed = L.EmbedID(
                vocab_size, 300, ignore_label=-1) # f_0
            self.embed_class = L.EmbedID(
                n_class, 300, ignore_label=-1)
            self.conv1 = L.ConvolutionND(1, None, n_class, filter_size, 1, filter_size//2) # f_1
            self.fc2 = L.Linear(300, n_class) # f_2

    def __call__(self, x):
        V = self.embed(x)
        V_norm = F.normalize(V.transpose(0, 2, 1), axis=1)
        C = self.embed_class.W
        C_norm = F.normalize(C, axis=1)
        G = F.matmul(F.broadcast_to(C_norm, (V_norm.shape[0], C_norm.shape[0], C_norm.shape[1])), V_norm)
        u = F.relu(self.conv1(G))
        m = F.maxout(u, pool_size=self.n_class, axis=1)
        beta = F.softmax(m, axis=2)
        z = F.sum((V * F.broadcast_to(beta.transpose(0,2,1), V.shape)), axis=1)
        z = self.fc2(F.dropout(z))
        z = F.sigmoid(z)
        z_class = self.fc2(F.dropout(C))
        out = F.concat([z, z_class], axis=0)
        return out    

In [18]:
##F2の処理で全結合の前にカーネルサイズの違う５つの畳み込み層を用いる
##concatした複数のzをmaxoutして全結合処理
class LEAM(chainer.Chain):
    
    def __init__(self, vocab_size, n_class=6,filter_size=[1,3,5,7,9]):
        super(LEAM, self).__init__()
        self.n_class = n_class
        with self.init_scope():
            self.embed = L.EmbedID(vocab_size, 300, ignore_label=-1) # f_0
            self.embed_class = L.EmbedID(n_class, 300, ignore_label=-1)
            self.conv = L.ConvolutionND(1, None, n_class, filter_size[0], 1, filter_size[0]//2)
            self.conv1 = L.ConvolutionND(1, None, 1, filter_size[0], 1, filter_size[0]//2) # f_1
            self.conv2 = L.ConvolutionND(1, None, 1, filter_size[1], 1, filter_size[1]//2)
            self.conv3 = L.ConvolutionND(1, None, 1, filter_size[2], 1, filter_size[2]//2)
            self.conv4 = L.ConvolutionND(1, None, 1, filter_size[3], 1, filter_size[3]//2)
            self.conv5 = L.ConvolutionND(1, None, 1, filter_size[4], 1, filter_size[4]//2)
            self.fc2 = L.Linear(300, n_class) # f_2

    def __call__(self, x):
        V = self.embed(x)
        V_norm = F.normalize(V.transpose(0, 2, 1), axis=1)
        C = self.embed_class.W
        C_norm = F.normalize(C, axis=1)
        G = F.matmul(F.broadcast_to(C_norm, (V_norm.shape[0], C_norm.shape[0], C_norm.shape[1])), V_norm)
        u = F.relu(self.conv(G))
        u = F.reshape(u,(V.shape[0],u.shape[1],u.shape[2]))
        m = F.maxout(u, pool_size=self.n_class, axis=1)
        beta = F.softmax(m, axis=2)
                           
        z = F.sum((V * F.broadcast_to(beta.transpose(0,2,1), V.shape)), axis=1)
        
        z = F.reshape(z,(z.shape[0],1,z.shape[1]))
        z1 = F.relu(self.conv1(z))
        z2 = F.relu(self.conv2(z))
        z3 = F.relu(self.conv3(z))
        z4 = F.relu(self.conv4(z))
        z5 = F.relu(self.conv5(z))
        z  = F.maxout(F.concat([z1,z2,z3,z4,z5],axis=1),pool_size=5,axis=1)
        
        z = F.reshape(z,(z.shape[0],z.shape[2]))
        z = self.fc2(F.dropout(z))
        z = F.sigmoid(z)
        z_class = self.fc2(F.dropout(C))
        out = F.concat([z, z_class], axis=0)
        return out

In [11]:
##絵文字の意味も計算して２つのbetaから学習を行った
class LEAM(chainer.Chain):    
    def __init__(self, vocab_size, n_class=6,filter_size=7):
        super(LEAM, self).__init__()
        self.n_class = n_class
        with self.init_scope():
            self.embed = L.EmbedID(vocab_size, 300, ignore_label=-1) # f_0
            self.embed_class1 = L.EmbedID(n_class, 300, ignore_label=-1)
            self.embed_class2 = L.EmbedID(n_class, 300, ignore_label=-1)
            self.conv1 = L.ConvolutionND(1, None, n_class, filter_size, 1, filter_size//2) # f_1
            self.fc1 = L.Linear(None,305)
            self.fc2 = L.Linear(300, n_class) # f_2

    def __call__(self, x):
        V = self.embed(x)
        V_norm = F.normalize(V.transpose(0, 2, 1), axis=1)
        
        C1 = self.embed_class1.W
        C_norm = F.normalize(C1, axis=1)
        G = F.matmul(F.broadcast_to(C_norm, (V_norm.shape[0], C_norm.shape[0], C_norm.shape[1])), V_norm)
        u = F.relu(self.conv1(G))
        m = F.maxout(u, pool_size=self.n_class, axis=1)
        beta1 = F.softmax(m, axis=2)
        
        C = self.embed_class2.W
        C_norm = F.normalize(C, axis=1)
        G = F.matmul(F.broadcast_to(C_norm, (V_norm.shape[0], C_norm.shape[0], C_norm.shape[1])), V_norm)
        u = F.relu(self.conv1(G))
        m = F.maxout(u, pool_size=self.n_class, axis=1)
        beta2 = F.softmax(m, axis=2)
        
        beta = F.maxout(F.concat([beta1,beta2]),pool_size=2,axis=1)
        z = F.sum((V * F.broadcast_to(beta.transpose(0,2,1), V.shape)), axis=1)
        z = self.fc2(F.dropout(z))
        z = F.sigmoid(z)
        z_class = self.fc2(F.dropout(C1))
        out = F.concat([z, z_class], axis=0)
        return out 

In [3]:
class cnn(chainer.Chain):
    
    def __init__(self, vocab_size, n_class=6,filter_size=[1,3,5,7,9]):
        super(cnn, self).__init__()
        self.n_class = n_class
        self.filter_size = filter_size
        with self.init_scope():
            self.embed = L.EmbedID(
                vocab_size, 300, ignore_label=-1) # f_0
            self.conv1 = L.ConvolutionND(1, None, n_class, filter_size[0], 1, filter_size[0]//2) # f_1
            self.conv2 = L.ConvolutionND(1, None, n_class, filter_size[1], 1, filter_size[1]//2)
            self.conv3 = L.ConvolutionND(1, None, n_class, filter_size[2], 1, filter_size[2]//2)
            self.conv4 = L.ConvolutionND(1, None, n_class, filter_size[3], 1, filter_size[3]//2)
            self.conv5 = L.ConvolutionND(1, None, n_class, filter_size[4], 1, filter_size[4]//2)
#             self.fc1 = L.Linear(None, 300)
            self.fc2 = L.Linear(None, n_class) # f_2
#             self.bnorm1=L.BatchNormalization(6)

    def __call__(self, x):
        V = self.embed(x)
        
#         u1 = self.bnorm1(F.relu(self.conv1(V)))       
#         u2 = self.bnorm1(F.relu(self.conv2(V)))
#         u3 = self.bnorm1(F.relu(self.conv3(V)))
#         u4 = self.bnorm1(F.relu(self.conv4(V)))
#         u5 = self.bnorm1(F.relu(self.conv5(V)))
        
        u1 = F.relu(self.conv1(V))       
        u2 = F.relu(self.conv2(V))
        u3 = F.relu(self.conv3(V))
        u4 = F.relu(self.conv4(V))
        u5 = F.relu(self.conv5(V))
        
        m1 = F.max_pooling_1d(u1,self.filter_size[0],1,pad=self.filter_size[0]//2)
        m2 = F.max_pooling_1d(u2,self.filter_size[1],1,pad=self.filter_size[1]//2)
        m3 = F.max_pooling_1d(u3,self.filter_size[2],1,pad=self.filter_size[2]//2)
        m4 = F.max_pooling_1d(u4,self.filter_size[3],1,pad=self.filter_size[3]//2)
        m5 = F.max_pooling_1d(u5,self.filter_size[4],1,pad=self.filter_size[4]//2)
        
        z = F.concat([m1,m2,m3,m4,m5],axis=1)
        
        m = F.reshape(z,(z.shape[0],300*z.shape[1]))
        m = F.normalize(m, axis=1)
        m = self.fc2(F.dropout(m))
        out = F.sigmoid(m)
        return out

In [ ]:
class LEAM(chainer.Chain):
    
    def __init__(self, vocab_size, n_class=6,filter_size=[3,5,7]):
        super(LEAM, self).__init__()
        self.n_class = n_class
        self.filter_size = filter_size
        with self.init_scope():
            self.conv2 = L.ConvolutionND(1, None, 305, filter_size[0], 1, filter_size[0]//2) # f_1
            self.embed = L.EmbedID(
                vocab_size, 300, ignore_label=-1) # f_0
            self.embed_class = L.EmbedID(
                n_class, 300, ignore_label=-1)
            self.conv1 = L.ConvolutionND(1, None, n_class, 55, 1, 55//2) # f_1
            self.fc2 = L.Linear(300, n_class) # f_2

    def __call__(self, x):
        V = self.embed(x)
        V_norm = F.normalize(V.transpose(0, 2, 1), axis=1)
        C = self.embed_class.W
        C_norm = F.normalize(C, axis=1)
        G = F.matmul(F.broadcast_to(C_norm, (V_norm.shape[0], C_norm.shape[0], C_norm.shape[1])), V_norm)
        u = F.relu(self.conv1(G))
        m = F.maxout(u, pool_size=self.n_class, axis=1)
        beta = F.softmax(m, axis=2)
        
        z = (V * F.broadcast_to(beta.transpose(0,2,1), V.shape))
        z2 = F.relu(self.conv2(z))
        m2 = F.max_pooling_1d(z2,self.filter_size[0],1,pad=self.filter_size[0]//2)
        z = F.sum(m2, axis=1)
        
        z = self.fc2(F.dropout(z))
        z = F.sigmoid(z)
        z_class = self.fc2(F.dropout(C))
        out = F.concat([z, z_class], axis=0)
        return out

In [91]:
def lossfun_multi(z, t):
    with cuda.get_device(z):
            z_class = z[-n_class:]
            z = z[:-n_class]
    t_class = Variable(cp.array(range(n_class)))
    loss = - (F.mean(t * F.log(z) + (1-t)*F.log1p(-z))) / z.shape[0] + F.softmax_cross_entropy(z_class, t_class)
    return loss

In [92]:
def auc_fun(z, t):
    aucs = []
    with cuda.get_device(z):
        z = z[:-n_class]
    t = chainer.cuda.to_cpu(t)
    z = chainer.cuda.to_cpu(z.data)
    for i in range(6):
        fpr, tpr, thresholds = metrics.roc_curve(t[::,i], z[::,i]) 
        aucs.append(metrics.auc(fpr, tpr))
#     print(aucs)
    return Variable(np.array(sum(aucs)/len(aucs)))

In [82]:
len(word2id)

40615

In [93]:
model = LEAM(len(word2id), 6)
serializers.load_npz('./paper_default/predict/leam_30kdata', model)

gpu_id = 0
if gpu_id >= 0:
    model.to_gpu(gpu_id)
model = L.Classifier(model, lossfun=lossfun_multi, accfun=auc_fun)    
model.to_cpu()

In [94]:
def Bscore(true_y,pred_y):
    lists = []
    for i in range(6):
#         lists.append(sum(sum((true_y - pred_y)**2)/(6*pred_y.shape[0])))
        lists.append(sum((true_y[:,i]-pred_y[:,i].data)**2)/pred_y.shape[0])
    print(lists)    
    return lists  

def aucScore(true_y,pred_y):
    return plot_graph(true_y,pred_y)

def plot_graph(y_true,y_pred):
    from sklearn import metrics
    import matplotlib.pyplot as plt
    import numpy as np
    # FPR, TPR(, しきい値) を算出
#     plt.plot()
    lists = []
    for i in range(6):
        fpr, tpr, thresholds = metrics.roc_curve(y_true[:,i], y_pred[:,i].data)
        # ついでにAUCも
        auc = metrics.auc(fpr, tpr)
        lists.append(auc)
    print(lists)    
    return lists
def precision_recall_F(true_y,pred_y):
    from sklearn.metrics import f1_score
    lists = []
    pred_y = (pred_y.data >= 0.3).astype(np.int32)
    for i in range(6):
        lists.append(f1_score(true_y[:,i], pred_y[:,i]))
    print(lists)    
    return lists    

In [172]:
div = 5000
count = test_x.shape[0]//div
Bscore_list = []
auc_list = []
fscore_list = []
for i in range(count):
    pred_y = model.predictor(np.array(test_x[i*div:(i+1)*div]))
    true_y = test_y[i*div:(i+1)*div]
    Bscore_list.append(Bscore(true_y,pred_y[:-6]))
    auc_list.append(aucScore(true_y,pred_y[:-6]))
    fscore_list.append(precision_recall_F(true_y,pred_y[:-6]))    
    pred_y_ = 0
    true_y = 0

Bscore_list = np.array(Bscore_list)
auc_list = np.array(auc_list)
fscore_list = np.array(fscore_list)
B_ave = []
auc_ave = []
f_ave = []
for i in range(6):
    B_ave.append(sum(Bscore_list[:,i])/count)
    auc_ave.append(sum(auc_list[:,i])/count)
    f_ave.append(sum(auc_list[:,i])/count)

print("-"*50)
print(B_ave)
print(auc_ave)
print(f_ave)

[0.06474565679124006, 0.06989401736404631, 0.0748214104641706, 0.07289330266474321, 0.08242542821848374, 0.06500949355470062]
[0.8410300201960735, 0.75333279491309, 0.741512436593273, 0.7496355266797504, 0.7629146917354386, 0.8230344511729683]
[0.4159613059250303, 0.2752043596730245, 0.16993464052287582, 0.2869198312236287, 0.282147315855181, 0.4047337278106509]
[0.0629672521726786, 0.07077704992681766, 0.07472265173016271, 0.07279143217641658, 0.07722564998554764, 0.05937533269720517]
[0.8611653641866662, 0.7519015957944762, 0.7304663131628196, 0.7759069840708439, 0.7582040249866308, 0.8387461344863331]
[0.4709976798143852, 0.22865412445730826, 0.15257048092868988, 0.2662889518413598, 0.27456258411843876, 0.4332493702770781]
[0.06706490411363084, 0.06713661611882756, 0.06848188212257161, 0.07261506635626025, 0.08241515960360749, 0.06330339783445495]
[0.8330266005738808, 0.7582940942014046, 0.7421179668065365, 0.7740642605254436, 0.7707692816000176, 0.8146005872018125]
[0.4092009685230

In [4]:
class cnn(chainer.Chain):
    
    def __init__(self, vocab_size, n_class=6,filter_size=[1,3,5,7,9]):
        super(cnn, self).__init__()
        self.n_class = n_class
        self.filter_size = filter_size
        with self.init_scope():
            self.embed = L.EmbedID(
                vocab_size, 300, ignore_label=-1) # f_0
            self.conv1 = L.ConvolutionND(1, None, 305, filter_size[0], 1, filter_size[0]//2) # f_1
            self.conv2 = L.ConvolutionND(1, None, 305, filter_size[1], 1, filter_size[1]//2)
            self.conv3 = L.ConvolutionND(1, None, 305, filter_size[2], 1, filter_size[2]//2)
            self.conv4 = L.ConvolutionND(1, None, 305, filter_size[3], 1, filter_size[3]//2)
            self.conv5 = L.ConvolutionND(1, None, 305, filter_size[4], 1, filter_size[4]//2)
#             self.fc1 = L.Linear(None, 300)
            self.fc2 = L.Linear(None, n_class) # f_2
#             self.bnorm1=L.BatchNormalization(6)

    def __call__(self, x):
        V = self.embed(x)
        
#         u1 = self.bnorm1(F.relu(self.conv1(V)))       
#         u2 = self.bnorm1(F.relu(self.conv2(V)))
#         u3 = self.bnorm1(F.relu(self.conv3(V)))
#         u4 = self.bnorm1(F.relu(self.conv4(V)))
#         u5 = self.bnorm1(F.relu(self.conv5(V)))
        
        u1 = F.relu(self.conv1(V))       
        u2 = F.relu(self.conv2(V))
        u3 = F.relu(self.conv3(V))
        u4 = F.relu(self.conv4(V))
        u5 = F.relu(self.conv5(V))
        
        m1 = F.max_pooling_1d(u1,self.filter_size[0],1,pad=self.filter_size[0]//2)
        m2 = F.max_pooling_1d(u2,self.filter_size[1],1,pad=self.filter_size[1]//2)
        m3 = F.max_pooling_1d(u3,self.filter_size[2],1,pad=self.filter_size[2]//2)
        m4 = F.max_pooling_1d(u4,self.filter_size[3],1,pad=self.filter_size[3]//2)
        m5 = F.max_pooling_1d(u5,self.filter_size[4],1,pad=self.filter_size[4]//2)
        
        z = F.concat([m1,m2,m3,m4,m5],axis=1)
        
        m = F.reshape(z,(z.shape[0],300*z.shape[1]))
        m = F.normalize(m, axis=1)
        m = self.fc2(F.dropout(m))
        out = F.sigmoid(m)
        return out

# テキストから予測

In [9]:
class LEAM(chainer.Chain):
    
    def __init__(self, vocab_size, n_class=6,filter_size=[3,5,7]):
        super(LEAM, self).__init__()
        self.n_class = n_class
        self.filter_size = filter_size
        with self.init_scope():
            self.conv2 = L.ConvolutionND(1, None, 305, filter_size[0], 1, filter_size[0]//2) # f_1
            self.conv3 = L.ConvolutionND(1, None, 305, filter_size[1], 1, filter_size[1]//2)
            self.conv4 = L.ConvolutionND(1, None, 305, filter_size[2], 1, filter_size[2]//2)
            self.embed = L.EmbedID(
                vocab_size, 300, ignore_label=-1) # f_0
            self.embed_class = L.EmbedID(
                n_class, 300, ignore_label=-1)
            self.conv1 = L.ConvolutionND(1, None, n_class, 55, 1, 55//2) # f_1
            self.fc2 = L.Linear(300, n_class) # f_2

    def __call__(self, x):
        V = self.embed(x)
        V_norm = F.normalize(V.transpose(0, 2, 1), axis=1)
        C = self.embed_class.W
        C_norm = F.normalize(C, axis=1)
        G = F.matmul(F.broadcast_to(C_norm, (V_norm.shape[0], C_norm.shape[0], C_norm.shape[1])), V_norm)
        u = F.relu(self.conv1(G))
        m = F.maxout(u, pool_size=self.n_class, axis=1)
        beta = F.softmax(m, axis=2)        
        z = F.sum(V * F.broadcast_to(beta.transpose(0,2,1), V.shape), axis=1)
        z = self.fc2(F.dropout(z))
        z = F.sigmoid(z)
        z_class = self.fc2(F.dropout(C))
        out = F.concat([z, z_class], axis=0)
        return out


def auc_fun_cnn(z, t):
    aucs = []
    with cuda.get_device(z):
        z = z
    t = chainer.cuda.to_cpu(t)
    z = chainer.cuda.to_cpu(z.data)
    
    for i in range(6):
        fpr, tpr, thresholds = metrics.roc_curve(t[::,i], z[::,i]) 
        aucs.append(metrics.auc(fpr, tpr))
        
    return Variable(np.array(sum(aucs)/len(aucs)))

def lossfun_multi_cnn(z, t):
    with cuda.get_device(z):
        z = z
    loss = - (F.mean(t * F.log(z) + (1-t)*F.log1p(-z)))
    return loss

def pred(text,emoji_index,term_vec,word2id):
    m = MeCab.Tagger("-Owakati -d /home/machi52/.local/lib/mecab/dic/mecab-ipadic-neologd")
    index_emoji = {index:emoji for emoji, index in emoji_index.items()}
    model = LEAM(len(word2id), 6)
    serializers.load_npz('../eval_leam/a/model_iteration-7500', model)
    output = {}
    gpu_id = 0

    if gpu_id >= 0:
        model.to_gpu(gpu_id)
    model = L.Classifier(model, lossfun=lossfun_multi_cnn, accfun=auc_fun_cnn) 
    f = lambda x: np.pad(x, pad_width=(0, 305-len(x)), mode='constant', constant_values=-1)
    
    line = text
    buff = []
    for i,term in enumerate(m.parse(line).strip().split()):
        if word2id.get(term) != None:
            buff.append(word2id[term])
        elif re.match("|".join(list(emoji_index.keys())),term) != None:
            return term
            
    id_texts = np.array(list(map(f, [buff])), np.int32) 
    results = model.predictor(cp.array(id_texts))
    res = {index_emoji[i]:score for i,score in enumerate(results[0].data.tolist())}
    emojis = []
    scores = []
    for emoji, score in sorted(filter(lambda x:x[1]>0.001, res.items()), key=lambda x:x[1]*-1)[:20]:
        emojis.append(emoji)
        scores.append(score*100)   
    for i in range(len(emojis)):    
        output.update({emojis[i]:int(scores[i])}) 
#     print(output)    
    return "("+" ".join(["{0}:{1}%".format(key, value) for key,value in sorted(output.items(), key=lambda x: -x[1])]) +")"
def softmax(a):
    # 一番大きい値を取得
    c = np.max(a)
    # 各要素から一番大きな値を引く（オーバーフロー対策）
    y = a/c
    
    return y

In [5]:
term_vec    = pickle.loads(open('../paper_default/vectors/fastText/fast_vectors.pkl', 'rb').read())
emoji_index = pickle.loads(open('../../all_pkl/emoji_index.pkl', 'rb').read())
word2id = pickle.loads(open('../eval_leam/word2id_600k', 'rb').read())
with codecs.open("../hiro_text.txt","r","utf-8") as r:
    line = r.read()    

In [14]:
emoji_index 

{'😊': 0, '😡': 5, '😩': 2, '😭': 4, '😱': 1, '😵': 3}

In [10]:


lineText = re.split("20.*"+"\(.+\)",line)[-2]
lineText = re.sub("\n","",lineText)
lineText = re.sub("\t"," ",lineText)
lineText = re.split("[0-9]{2}:[0-9]{2}",lineText)

for text in lineText[1:]:
    text = text.split()
    if len(text[1]) <=3:
        continue
    else:
        print(text[0],text[1],pred(text[1],emoji_index,term_vec,word2id))
        print("-"*50)

まっちー おはよう (😊:88 😡:8 😩:4 😵:3 😱:2 😭:1)
--------------------------------------------------
ひろ きてる？ (😡:48 😱:17 😵:10 😊:10 😭:9 😩:5)
--------------------------------------------------
まっちー おっきな門がみえる！ (😭:16 😱:14 😵:13 😡:13 😊:8 😩:7)
--------------------------------------------------
ひろ 早く来いや〜 (😡:70 😩:9 😭:7 😵:6 😱:2 😊:1)
--------------------------------------------------


KeyboardInterrupt: 

In [10]:
text = input('テキストを入力してください：\n')
text = re.sub('😊|😵|😡|😭|😱|😩', '', text)
pred(text,emoji_index,term_vec,word2id)

テキストを入力してください：
 


'(😡:53% 😱:51% 😩:49% 😭:47% 😊:46% 😵:45%)'

In [7]:
line = ["アミバさんこんにちは😊","即来て即領海侵入しやがりました😡","年明けから飛ばしてきますね。","去年と同じパターンです。","こちらこそよろしくです👍"]
for text in line:
    a = text
    text = re.sub('😊|😵|😡|😭|😱|😩', '', text)  
    print("text: "+re.sub("\n","",a)+":label("+",".join(list(set(re.findall('😊|😵|😡|😭|😱|😩',a))))+")")
    print(pred(text,emoji_index,term_vec,word2id))
    print("-"*50)

text: アミバさんこんにちは😊:label(😊)
(😊:98% 😡:15% 😭:7% 😵:3% 😱:3% 😩:2%)
--------------------------------------------------
text: 即来て即領海侵入しやがりました😡:label(😡)
(😡:54% 😊:46% 😭:31% 😵:27% 😩:25% 😱:16%)
--------------------------------------------------
text: 年明けから飛ばしてきますね。:label()
(😊:63% 😵:53% 😱:28% 😭:23% 😩:22% 😡:3%)
--------------------------------------------------
text: 去年と同じパターンです。:label()
(😱:74% 😵:58% 😭:47% 😊:32% 😩:18% 😡:1%)
--------------------------------------------------
text: こちらこそよろしくです👍:label()
(😊:99% 😱:2% 😭:0% 😩:0% 😵:0% 😡:0%)
--------------------------------------------------


In [11]:
list_df = pd.DataFrame( columns=['text','true',"pred"] )
with codecs.open("./kensyouyou.txt","r","utf-8") as r:
    line = r.readlines()
    for text in line:
        test = re.sub('😊|😵|😡|😭|😱|😩', '', text)  
        
        text = re.sub("\n","",text)
        label = ",".join(list(set(re.findall('😊|😵|😡|😭|😱|😩',text))))
        pred_label = pred(test,emoji_index,term_vec,word2id)
        print(text)
        print("label: "+label)
        print(pred_label)
        print("-"*50)
        
        tmp_se = pd.Series( [ text,label,pred_label ], index=list_df.columns )
        list_df = list_df.append( tmp_se, ignore_index=True )

田中アナの文章いいな😊圭さんに対する尊敬と愛情が静かに熱く伝わってくる。今夜は見る！うん。「田中圭24時間テレビ」現場で見えた田中圭の「勉強力」　田中大貴アナ連載｜CoCoKARAnext（ココカラnext）
label: 😊
(😊:62% 😡:36% 😩:15% 😱:11% 😵:9% 😭:2%)
--------------------------------------------------
一通り読んだぜ🤔状況把握しました🥺既読つかなかったり連絡ないと不安になるよ😵遠距離だから余計にね…こっちばっかりなんか不安なのかな、とか思ったりしてね😭電話の件だって…電話するならするで自分に集中してほしいって思っちゃうよ🥺せっかくの電話なのにさ😭
label: 😭,😵
(😩:70% 😵:44% 😱:28% 😭:22% 😡:9% 😊:1%)
--------------------------------------------------
しろやオラァ！！！！！😡💢💢💢💢💢🔥
label: 😡
(😡:99% 😩:5% 😭:3% 😱:0% 😊:0% 😵:0%)
--------------------------------------------------
あーーーー共感😭😭💛💛
label: 😭
(😭:87% 😩:45% 😱:30% 😊:19% 😡:18% 😵:7%)
--------------------------------------------------
びっくりー⁉️😂今娘を🚗でおくっていたら…片側2車線の道を進行方向から逆走してくる軽自動車が〜😱😱こわ〜い💧お年寄りかなー😰⁉️今日は周りをしっかり見て気をつける日✨な気がします😂みなさんも気をつけてください💕
label: 😱
(😱:68% 😵:60% 😩:32% 😭:8% 😊:7% 😡:6%)
--------------------------------------------------
カラオケいきたいよく誰か近々いかん？😩😩
label: 😩
(😭:57% 😩:51% 😡:27% 😱:24% 😵:18% 😊:6%)
--------------------------------------------------
女の子だと、難しそうですね😊お母さんが、全てを知っている。なーんちゃって

In [67]:
list_df = pd.DataFrame( columns=['text','true',"pred"] )
with codecs.open("./kensyouyou.txt","r","utf-8") as r:
    line = r.readlines()
    for text in line:
        test = re.sub('😊|😵|😡|😭|😱|😩', '', text)  
        
        text = re.sub("\n","",text)
        label = ",".join(list(set(re.findall('😊|😵|😡|😭|😱|😩',text))))
        pred_label = pred(test,emoji_index,term_vec,word2id)
        print(text)
        print("label: "+label)
        print(pred_label)
        print("-"*50)
        
        tmp_se = pd.Series( [ text,label,pred_label ], index=list_df.columns )
        list_df = list_df.append( tmp_se, ignore_index=True )

田中アナの文章いいな😊圭さんに対する尊敬と愛情が静かに熱く伝わってくる。今夜は見る！うん。「田中圭24時間テレビ」現場で見えた田中圭の「勉強力」　田中大貴アナ連載｜CoCoKARAnext（ココカラnext）
label: 😊
(😊:47% 😵:15% 😭:11% 😱:8% 😩:8% 😡:5%)
--------------------------------------------------
一通り読んだぜ🤔状況把握しました🥺既読つかなかったり連絡ないと不安になるよ😵遠距離だから余計にね…こっちばっかりなんか不安なのかな、とか思ったりしてね😭電話の件だって…電話するならするで自分に集中してほしいって思っちゃうよ🥺せっかくの電話なのにさ😭
label: 😭,😵
(😩:38% 😭:23% 😵:16% 😡:16% 😱:8% 😊:1%)
--------------------------------------------------
しろやオラァ！！！！！😡💢💢💢💢💢🔥
label: 😡
(😡:93% 😭:2% 😩:1% 😊:0% 😵:0% 😱:0%)
--------------------------------------------------
あーーーー共感😭😭💛💛
label: 😭
(😭:69% 😩:26% 😱:8% 😵:5% 😊:3% 😡:1%)
--------------------------------------------------
びっくりー⁉️😂今娘を🚗でおくっていたら…片側2車線の道を進行方向から逆走してくる軽自動車が〜😱😱こわ〜い💧お年寄りかなー😰⁉️今日は周りをしっかり見て気をつける日✨な気がします😂みなさんも気をつけてください💕
label: 😱
(😱:56% 😵:46% 😊:15% 😩:8% 😭:7% 😡:0%)
--------------------------------------------------
カラオケいきたいよく誰か近々いかん？😩😩
label: 😩
(😩:48% 😭:18% 😵:14% 😡:11% 😱:8% 😊:5%)
--------------------------------------------------
女の子だと、難しそうですね😊お母さんが、全てを知っている。なーんちゃって😅
lab

In [27]:
list_df.to_csv("./kensyouyou.csv", index=False, encoding="utf-8", mode='a', header=False)

In [34]:
text = "一通り読んだぜ🤔状況把握しました🥺既読つかなかったり連絡ないと不安になるよ😵遠距離だから余計にね…こっちばっかりなんか不安なのかな、とか思ったりしてね😭電話の件だって…電話するならするで自分に集中してほしいって思っちゃうよ🥺せっかくの電話なのにさ😭"
    re.findall('😊|😵|😡|😭|😱|😩',text)

['😵', '😭', '😭']

In [49]:
import tweepy
import re
import time

C_KEY = "5VJqwPmqU43NZRGzEp3BMSPoz"
C_SECRET = "5uTonNvxYcK4LR3ho15x7da9BP83JyWppcsuUarkzJnrtfZMwh"
A_KEY = "2785997472-ghhk3LuE9L6nu7gOgYusQzz3ADXLnX4tw4CmjDb"
A_SECRET = "8tMOZtuVd0oLFJ9hhxiu5bEir1JCaJKZP33NhPJoZ6fZG"

auth = tweepy.OAuthHandler(C_KEY, C_SECRET)
auth.set_access_token(A_KEY, A_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

emotion=["😊","😵","😡","😭","😱","😩"]
# emotion=["😊","😊","😊","😊","😊","😊"]
max_id=""
print("ok")

texts = []
labels = []
pred_labels = []

for i,keyword in enumerate(emotion):
    api = tweepy.API(auth, wait_on_rate_limit=True)
    for status in api.search(q=keyword, lang='ja', result_type='recent',count=100,since_id=max_id):
        text = re.sub("\n","",status.text)
#         text = re.sub('😊|😵|😡|😭|😱|😩', '', text)
        text = re.sub("RT","",text)
        text = re.sub("@.*?\s","",text)
        text = re.sub("\shttp.*","",text)
        
        test = re.sub('😊|😵|😡|😭|😱|😩', '', text)  
        
        label = ",".join(list(set(re.findall('😊|😵|😡|😭|😱|😩',text))))
        pred_label = pred(test,emoji_index,term_vec,word2id)
        
        print(label,pred_label)
        texts.append(text)
        labels.append(label)
        pred_labels.append(pred_label)

ok
😊 (😭:51 😩:21 😊:17 😵:11 😱:6 😡:1)
😊 (😊:64 😵:14 😱:10 😭:8 😩:6 😡:0)
😊 (😊:64 😭:34 😵:8 😩:5 😱:3 😡:0)
😊 (😊:68 😭:17 😵:6 😱:5 😩:1 😡:0)
😊 (😊:66 😭:15 😩:14 😵:8 😱:7 😡:2)
😊 (😡:31 😭:22 😩:16 😊:12 😵:6 😱:3)
😊 (😊:69 😭:35 😩:6 😵:3 😱:3 😡:0)
😊 (😡:34 😊:29 😩:19 😵:14 😭:7 😱:3)
😊 (😊:88 😭:14 😡:2 😩:1 😵:0 😱:0)
😊 (😭:26 😵:24 😊:23 😱:22 😩:4 😡:2)
😊 (😊:86 😵:13 😭:9 😱:6 😩:2 😡:0)
😊 (😱:26 😵:24 😡:18 😩:14 😊:7 😭:5)
😊 (😵:41 😱:21 😩:20 😊:18 😭:10 😡:3)
😊 (😡:19 😱:17 😊:16 😭:16 😩:14 😵:13)
😊 (😊:69 😭:42 😵:4 😩:4 😱:2 😡:0)
😊 (😊:57 😭:43 😵:10 😱:5 😩:4 😡:0)
😊 (😊:90 😭:17 😵:0 😡:0 😱:0 😩:0)
😊 (😊:38 😵:36 😱:15 😩:11 😭:5 😡:0)
😊 (😊:73 😭:22 😩:2 😵:1 😡:0 😱:0)
😊 (😊:62 😭:21 😡:11 😩:5 😵:2 😱:1)
😊,😱 (😊:64 😭:18 😵:9 😱:5 😩:2 😡:0)
😊 (😊:53 😵:33 😱:18 😭:15 😩:9 😡:0)
😊 (😊:94 😭:5 😵:1 😡:0 😱:0 😩:0)
😊 (😩:34 😭:29 😵:11 😡:9 😊:5 😱:3)
😊,😵 (😵:32 😩:23 😊:21 😱:21 😭:20 😡:0)
😊 (😊:49 😡:23 😵:9 😩:9 😱:7 😭:4)
😊 (😊:95 😭:4 😵:0 😡:0 😱:0 😩:0)
😊 (😊:59 😭:14 😩:9 😵:5 😱:3 😡:1)
😊 (😊:53 😵:26 😱:15 😭:13 😩:5 😡:0)
😊 (😊:77 😭:15 😵:9 😩:4 😱:3 😡:0)
😊 (😭:42 😊:37 😱:15 😵:14 😩:9 😡:0)
😊 (😊:81 😭:19 😵:3 😩:3 😱:2 😡:0)
😊 

In [57]:
print(texts[13],labels[13],pred_labels[13])

やっくん頑張れ😊😊😊😊赤葦くんの顔すごい😅ばにさんのおいかーさんととびよは大人しめ☺️☺️ひょっとして明日の毎日夜久さんは公式イラストの魔改造黒尾さんが来ちゃったりしちゃったりするんだろうか|˙꒳​˙） 😊 (😡:19 😱:17 😊:16 😭:16 😩:14 😵:13)


In [59]:
list_df = pd.DataFrame( columns=['text','true',"pred"] )
for i in range(len(texts)):
    tmp_se = pd.Series( [ texts[i],labels[i],pred_labels[i] ], index=list_df.columns )
    list_df = list_df.append( tmp_se, ignore_index=True )

In [61]:
list_df.to_csv("./pred.csv", index=False, encoding="utf-8", mode='a', header=False)

In [62]:
for i in range(len(texts)):

    print(texts[i],labels[i],pred_labels[i])

この表情はいけませんでしたねぇ（意味深この時はなぜかテンション上がって撮れたしこちらこそすごくありがたかったよ😊しかし...ほんと綺麗な表情だ... 😊 (😭:51 😩:21 😊:17 😵:11 😱:6 😡:1)
薬なしにチャレンジですか😳スっと気持ちよく寝落ちしてゆっくり休めるといいですね🍀😊良い眠りが訪れますように🌸おやすみなさい(*˘꒳˘*) 😊 (😊:64 😵:14 😱:10 😭:8 😩:6 😡:0)
ありがとうございます😢😊もう少しの間、よろしくお願いしますね💖😌 😊 (😊:64 😭:34 😵:8 😩:5 😱:3 😡:0)
よいこさんー！ありがとうございます😊😊ピコ太郎回避したんですね！！おめですーー😆😆 😊 (😊:68 😭:17 😵:6 😱:5 😩:1 😡:0)
今日のSHOWROOMめっちゃおもしろかったな笑やっぱりメンバーといる時楽しそうだ😊 😊 (😊:66 😭:15 😩:14 😵:8 😱:7 😡:2)
かわいそうなのがかわいいです😊 😊 (😡:31 😭:22 😩:16 😊:12 😵:6 😱:3)
いえいえ✨私も紫音さんにはいつもお世話になってますー😍💖父親もアニメとか大好きなの！お互い理解のある親だから面白いね😊久々のしなやちゃんのコスプレ楽しみにしてるよー！承太郎さんとラブラブする～～～😘💕💕💕 😊 (😊:69 😭:35 😩:6 😵:3 😱:3 😡:0)
遊ぼ〜♪じゃんけんして負けたほうが一枚ずつ脱いでくって遊びしよっか😊 #peing #質問箱 😊 (😡:34 😊:29 😩:19 😵:14 😭:7 😱:3)
25いいねありがとう😊 😊 (😊:88 😭:14 😡:2 😩:1 😵:0 😱:0)
 地元の古本市場にドラゴンボールヒーローズの500円ガチャがあったので試しに一回だけ回してみました😊結果は破壊王ブロリー‼️今年はガチャ運がイイみたいです‼️(今だけね…)ちなみにﾓﾝｽﾄですが単発4回でソロモン２体GETしまし… 😊 (😭:26 😵:24 😊:23 😱:22 😩:4 😡:2)
 【#有超天シューター 発売記念企画】#チョメ上拓也『吸っ拓也』東京ドームシティラクーアでのリリイベありがとうございました😊💕明日は尼崎とBMシアターでイベントです👏お待ちしております‼️#吸っ拓也 #吸った #ライブ後にはしっ… 😊 (😊:86 😵:13 😭:9 😱